<a href="https://colab.research.google.com/github/Ulugbek9403/ml_edu/blob/master/Test2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Привет!

Перед началом прохождения убедитесь, что все настроено как надо. Следующая ячейка должна выполниться без ошибок:

После прохождения не забудьте **перезапустить ядро** (**restart the kernel**) (в панели меню выберите Kernel $\rightarrow$ Restart) и затем **запуск всех ячеек** (**run all cells**) (в панели меню выберите Cell $\rightarrow$ Run All).

Убедитесь, что заменили все такие места
```python
# YOUR CODE HERE
raise NotImplementedError()
```
и
```
"YOUR ANSWER HERE"
```
на свои решения и ответы.

Если не установлены какие-то пакеты, то посмотрите нужные версии в ноутбуках по регрессии.

Для прорешивания используйте среды с поддержкой формата Jupyter: [Google Colab](https://colab.research.google.com/), [Jupyter Notebook (Web)](https://jupyter.org/), [PyCharm](https://www.jetbrains.com/ru-ru/pycharm/), [DataSpell](https://www.jetbrains.com/ru-ru/dataspell/) или [VSCode](https://code.visualstudio.com/). Работа с этим форматом в обычном текстовом редакторе может сыграть злую шутку.

Не забывайте о следующих особенностях:
- 😉 Помимо указанных тестов после задания существуют и скрытые тесты, поэтому внимательнее читайте задание
- 😎 Обязательно проверяйте выполнимость на тестах, для проверки перезапустите ядро и запустите все ячейки заново!
- 😂 После скачивания ноутбуков достаточно заполнить места для ответов и отправить ноутбук обратно (сам ноутбук, ни в архиве, ни ссылкой на GitHub ни как-то еще принимать не будем, загружаете сам файл)
- 😈 Не рекомендуем удалять исходные ячейки, так как из-за этого могут не начислиться баллы =(
  - 👼 Если таки такое случилось, скачайте заново шаблон и перенесите в него свои решения
- 😛 Свой код пишите только в ячейках для решений - любые дописывания в остальных ячейках (с текстом или тестами) могут не попасть в проверку (например, `import` вне ячейки для решения)


---

Это третья контрольная работа с некоторыми вопросами по теме классификации в машинном обучении!

In [125]:
import numpy as np
print(f"numpy version: {np.__version__}")
import pandas as pd
print(f"pandas version: {pd.__version__}")
# Just to check that it exists
import sklearn
print(f"scikit-learn version: {sklearn.__version__}")

from sklearn.datasets import load_iris, load_linnerud
from sklearn.metrics import classification_report, confusion_matrix

from typing import Union

RANDOM_SEED = 123
np.random.seed(RANDOM_SEED)

numpy version: 1.26.4
pandas version: 2.2.2
scikit-learn version: 1.5.2


### Задание 1

У вас имеется следующий код и разработчик дает следующее описание:

"Я сделал код для датасета linnerud, который выбирает примеры со значением прыжков меньше 80. Написал как смог =)"

```python
valid_data = []
for i in range(data.shape[0]):
    for j in range(data.shape[1]):
        if j == 2:
            row = data.iloc[i]
            value = row[2]
            if value < 80:
                valid_data.append(row.name)
```

Ваша задача - написать код, который выбирает примеры с количеством приседаний от 120 до 250 (не включительно). Для реализации необходимо использовать pandas.

In [126]:
def extract_average_situps_indexes(data: pd.DataFrame) -> list:
    # Фильтруем строки по значению приседаний и возвращаем индексы
    return data[(data['Situps'] > 120) & (data['Situps'] < 250)].index.tolist()





In [127]:
features, target = load_linnerud(return_X_y=True, as_frame=True)
indexes = extract_average_situps_indexes(features)
np.testing.assert_array_equal(indexes, [0, 4, 7, 8, 11, 12, 15, 17, 18])


### Задание 2

Напишите класс классификатора, который проверяет значение задаваемой колонки (индекса по порядку) (anomaly_col_idx) и для примеров со значением больше заданного порога (threshold) **по абсолютному значению** записывает в предсказание "Anomaly!". Для предсказания используйте LogisticRegression и не забудьте зафиксировать random_state (для этого есть RANDOM_SEED переменная)!


In [128]:
class ClassifierWithAnomalyDetection:
    def __init__(self, anomaly_col_idx: int, threshold: float):
        self.anomaly_col_idx = anomaly_col_idx
        self.threshold = threshold
        from sklearn.linear_model import LogisticRegression # Importing here to avoid import error
        self.model = LogisticRegression(random_state=RANDOM_SEED)

    def fit(self, X_train: pd.DataFrame, y_train: pd.Series):
        self.model.fit(X_train, y_train)

    def predict(self, X_test: pd.DataFrame):  # Indentation corrected to align with other methods
        predictions = self.model.predict(X_test)
        anomaly_conditions = X_test.iloc[:, self.anomaly_col_idx].abs() > self.threshold
        for i in range(len(predictions)):
            if anomaly_conditions.iloc[i]:
                predictions[i] = "Anomaly!"
        return predictions

In [129]:
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split

# Prepare data for testing
X_data, y_data = make_classification(
    n_samples=100,
    n_features=2,
    n_redundant=0,
    scale=[1, 10],
    random_state=RANDOM_SEED)

train_columns = ["Feature_1", "Feature_2"]
target_column = "target"

df = pd.DataFrame(X_data, columns=train_columns)
df[target_column] = pd.Series(y_data).map({0: "bad", 1: "good"})

train_df, test_df = train_test_split(df, test_size=0.3, random_state=RANDOM_SEED)

classifier = ClassifierWithAnomalyDetection(anomaly_col_idx=0, threshold=18)

classifier.fit(train_df[train_columns], train_df[target_column])
y_pred = classifier.predict(test_df[train_columns])
y_true = test_df[target_column]

assert "Anomaly!" in y_pred
assert "bad" in y_pred
assert "good" in y_pred

np.testing.assert_equal(len(y_pred), len(test_df))

### Задание 3

Перепешите реализацию кода с использованием pandas:

```python
df_1_data = df_main.values
df_2_data = df_noise.values

new_column = []
for i in range(df_1_data.shape[0]):
    for j in range(df_1_data.shape[1]):
        if j == 0:
            for k in range(df_2_data.shape[0]):
                for m in range(df_2_data.shape[1]):
                    if m == 0:
                        if df_1_data[i][j] == df_2_data[k][m]:
                            for n in range(df_2_data.shape[1]):
                                if n == 1:
                                    new_column.append(df_2_data[k][n])
df_main = df_main.copy()
df_main['has_noise'] = new_column

return df_main
```

In [130]:
def append_noise_column(df_main: pd.DataFrame, df_noise: pd.DataFrame) -> pd.DataFrame:
    df_with_noise = pd.merge(df_main, df_noise, on="id", how="left")
    return df_with_noise
    raise NotImplementedError()


In [131]:
df_main = pd.DataFrame(
    [
        [1, 1.5], [1, 2.3],
        [2, 0.7], [2, 1.6], [2, 3.4],
        [3, 0.5], [3, 0.5], [3, 0.5]
    ],
    columns=["id", "value"]
)
df_noise = pd.DataFrame(
    [
        [1, True], [2, False], [3, False]
    ],
    columns=["id", "has_noise"]
)

df_with_noise = append_noise_column(df_main, df_noise)

np.testing.assert_array_equal(df_with_noise.columns, ["id", "value", "has_noise"])

np.testing.assert_equal(df_with_noise.loc[0, "has_noise"], True)
np.testing.assert_equal(df_with_noise.loc[4, "has_noise"], False)
np.testing.assert_equal(df_with_noise.loc[7, "has_noise"], False)


### Задание 4

Найдите индексы строк, где колонка делится на n_div без остатка. Например, значения wine_serving на 3.

In [132]:
def get_n_divisible_indexes(df: pd.DataFrame, column_name: str, n_div: int) -> list:
    divisible_indexes = df[df[column_name] % n_div == 0].index.tolist()
    return divisible_indexes
    raise NotImplementedError()

In [133]:
features, target = load_linnerud(return_X_y=True, as_frame=True)

indexes = get_n_divisible_indexes(features, "Jumps", 5)
np.testing.assert_array_equal(indexes, [0,  1,  7,  8,  9, 11, 12, 13, 15, 16, 17])


### Задание 5

Напишите функцию, которая строит confusion matrix (true ~ rows, pred ~ cols) по заданным классам по осям. Остальные классы скидываются в объединенный класс "Другое".

Например, имеем:

```python
y_true = [0, 1, 1, 2, 2, 3, 1, 1]
y_pred = [1, 1, 1, 3, 2, 3, 1, 0]
```

Мы задаем целевые классы

```python
true_classes = [0, 1]
pred_classes = [1, 2]
```

Должна получиться матрица (DataFrame) с соответствующими именами строк и колонок. Порядок тоже должен сохраняться по задаваемому списку классов.

|    |  1 | 2  | Другое |
|---:|:--:|:--:|:--:  |
| 0  | 1  | 0  |    0    |
| 1  | 3  | 0  |   1     |
|Другое| 0 | 1   |   2     |


In [134]:
def custom_confusion_matrix(y_true: list, y_pred: list, true_classes: list, pred_classes:list) -> pd.DataFrame:
    conf_matrix = np.zeros((len(true_classes) + 1, len(pred_classes) + 1), dtype=int)
    for true_label, pred_label in zip(y_true, y_pred):
        if true_label in true_classes:
            true_index = true_classes.index(true_label)
        else:
            true_index = len(true_classes)

        if pred_label in pred_classes:
            pred_index = pred_classes.index(pred_label)
        else:
            pred_index = len(pred_classes)
        conf_matrix[true_index, pred_index] += 1
    true_classes.append("Другое")
    pred_classes.append("Другое")
    df = pd.DataFrame(conf_matrix, index=true_classes, columns=pred_classes)
    return df

In [135]:
y_true = [0, 1, 1, 2, 2, 3, 1, 1]
y_pred = [1, 1, 1, 3, 2, 3, 1, 0]

true_classes = [0, 1]
pred_classes = [1, 2]

conf = custom_confusion_matrix(y_true, y_pred, true_classes, pred_classes)
assert isinstance(conf, pd.DataFrame)
np.testing.assert_array_equal(conf.columns, np.array([1, 2, "Другое"], dtype=object))
np.testing.assert_array_equal(conf.index, np.array([0, 1, "Другое"], dtype=object))
np.testing.assert_equal(conf.loc[1, 1], 3)
np.testing.assert_equal(conf.loc["Другое", 2], 1)

In [136]:
# Another test
y_true = [0, 1, 1, 2, 2, 3, 1, 1, 2, 1, 0]
y_pred = [1, 1, 1, 3, 2, 3, 1, 0, 4, 1, 2]

true_classes = [3, 0]
pred_classes = [1, 2]

conf = custom_confusion_matrix(y_true, y_pred, true_classes, pred_classes)
np.testing.assert_array_equal(conf, [[0, 0, 1], [1, 1, 0], [4, 1, 3]])



### Задание 6

Напишите функцию, которая принимает вектор предсказаний и возвращает индексы примеров, в которых модель совершила ошибки. Обратите внимание, что входной вектор может быть как обычным list или tuple, так и pd.Series. Вернуть необходимо индексы (index) из исходного y_true.

In [137]:
def get_error_indexes(y_true: pd.Series, y_pred: Union[list, tuple, pd.Series]) -> pd.Index:
    """
    Возвращает индексы примеров, где модель сделала ошибку.

    Parameters:
        y_true (pd.Series): Вектор истинных значений.
        y_pred (Union[list, tuple, pd.Series]): Вектор предсказанных значений.

    Returns:
        pd.Index: Индексы примеров с ошибочными предсказаниями.
    """
    # Преобразование y_pred в pd.Series, если это список или кортеж
    if not isinstance(y_pred, pd.Series):
        y_pred = pd.Series(y_pred, index=y_true.index)

    # Вычисление индексов ошибок
    error_indexes = y_true.index[y_true != y_pred]

    return error_indexes
    raise NotImplementedError()

In [138]:
X, y = load_iris(as_frame=True, return_X_y=True)

y_test = y.iloc[50:60]
y_pred = [0, 0, 1, 1, 1, 0, 0, 0, 1, 1]

err_idxs = get_error_indexes(y_test, y_pred)
np.testing.assert_array_equal(err_idxs, [50, 51, 55, 56, 57])


In [139]:
X, y = load_iris(as_frame=True, return_X_y=True)

y_test = y.iloc[70:80]
y_pred = pd.Series([0, 1, 0, 1, 0, 1, 1, 0, 0, 1])

err_idxs = get_error_indexes(y_test, y_pred)

np.testing.assert_array_equal(err_idxs, [70, 72, 74, 77, 78])


ValueError: Can only compare identically-labeled Series objects

### Задание 7

Напишите функцию, которая по переданным `y_true, y_pred` строит стандартный `classification_report`, вычисляет `confusion_matrix` и обогащает данные из `classification_report` столбцом `specificity`. Столбец необходимо посчитать для каждого класса, а также `macro` и `weighted` значения.
Используйте готовые реализации там, где это возможно.

In [140]:
def extend_classification_report(y_true: np.ndarray, y_pred: np.ndarray, class_labels) -> pd.Index:
    import pandas as pd
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix

def extend_classification_report(y_true: list, y_pred: list, class_labels: list) -> pd.DataFrame:
    """
    Расширяет стандартный classification_report, добавляя столбец specificity.

    Parameters:
        y_true (list): Истинные метки классов.
        y_pred (list): Предсказанные метки классов.
        class_labels (list): Список всех меток классов.

    Returns:
        pd.DataFrame: Расширенный отчет с добавленным столбцом specificity.
    """
    # Генерация классификационного отчета в виде словаря
    report = classification_report(y_true, y_pred, target_names=class_labels, output_dict=True)

    # Вычисление матрицы ошибок
    cm = confusion_matrix(y_true, y_pred, labels=class_labels)

    # Вычисление специфичности для каждого класса
    specificity = {}
    for i, label in enumerate(class_labels):
        tn = cm.sum() - (cm[i, :].sum() + cm[:, i].sum() - cm[i, i])
        fp = cm[:, i].sum() - cm[i, i]
        specificity[label] = tn / (tn + fp) if (tn + fp) > 0 else 0.0

    # Добавляем macro avg и weighted avg для специфичности
    specificity['macro avg'] = np.mean(list(specificity.values()))
    weights = [sum(np.array(y_true) == label) for label in class_labels]
    specificity['weighted avg'] = np.average(list(specificity.values()), weights=weights)

    # Преобразование отчета в DataFrame
    df = pd.DataFrame(report).transpose()

    # Добавление столбца specificity
    df['specificity'] = pd.Series(specificity)

    return df

    raise NotImplementedError()

In [141]:
class_labels = ['yellow', 'red', 'white']
y_true = ['yellow', 'white', 'white', 'red', 'red', 'white', 'white', 'white', 'yellow', 'yellow', 'red', 'red']
y_pred = ['white', 'white', 'white', 'red', 'white', 'yellow', 'red', 'white', 'red', 'red', 'red', 'white']

df = extend_classification_report(y_true, y_pred, class_labels)

np.testing.assert_array_equal(df.index, ['yellow', 'red', 'white', 'accuracy', 'macro avg', 'weighted avg'])

np.testing.assert_almost_equal(df['specificity']['yellow'], 0.833)
np.testing.assert_almost_equal(df['specificity']['red'], 0.5)
np.testing.assert_almost_equal(df['specificity']['white'], 0.4)

np.testing.assert_almost_equal(df['specificity']['macro avg'], 0.578)
np.testing.assert_almost_equal(df['specificity']['weighted avg'], 0.542)


TypeError: Axis must be specified when shapes of a and weights differ.